In [6]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from torch import tensor
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import mean_squared_error
import random
import os
import matplotlib.pyplot as plt
#from numba import jit
import pickle
from scipy.interpolate import interp1d
from torch.utils.data import DataLoader, random_split
import torch
from torchsummary import summary
import seaborn as sns
import sys
import torch.nn.functional as F
import pywt
from sklearn.preprocessing import MinMaxScaler
from torch.cuda import FloatTensor

# Req for package
sys.path.append("../")
from SkinLearning.Utils.NN import train, test, DEVICE, getParameterLoss, set_seed
from SkinLearning.Utils.Dataset import getDataset, getSplit
from SkinLearning.Utils.Plotting import plotParameterBars


torch.backends.cudnn.benchmark = True

In [15]:
set_seed()

In [129]:
# Remove one FC LAyer
class MultiLSTM(nn.Module):
    def __init__(self, hidden_size=256, single_fc=True, out="f_hidden"):
        super(MultiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.out = out
        
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=5, padding=1, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        
            nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        self.cnn2 = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=5, padding=1, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        
            nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        self.lstm = nn.LSTM(15, hidden_size, batch_first=True, num_layers=1)
    
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256 , 128),
            nn.ReLU(),
            nn.Linear(128, 64), 
            nn.ReLU(),
            nn.Linear(64, 6),
        )

    def forward(self, x):
        batch_size = x.shape[0]
        x1 = x[:, 0, :].reshape(batch_size, 1, -1)
        x2 = x[:, 1, :].reshape(batch_size, 1, -1)
        
        x1 = self.cnn(x1)
        x2 = self.cnn(x2)
        
        #x = torch.cat([x1, x2], dim=2)

        
        #h0 = torch.zeros(1, batch_size, 256).to(x.device)
        #c0 = torch.zeros((1, batch_size, self.hidden_size)).to(x.device)
        o1, (h, c) = self.lstm(x1)
        o2, (h2, c) = self.lstm(x2)
        
        
        x = torch.concat([h[-1], h2[-1]], dim=1)
        #x = o1
        x = x.reshape(batch_size, -1)
        x = self.fc(x)
        
        return x

In [3]:
dataset, scaler = getDataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:10<00:00, 216.30it/s]


In [41]:
train_loader, test_loader = getSplit(dataset)

In [130]:
lstm = MultiLSTM()

In [131]:
test(test_loader, lstm, scaler)

100%|██████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 26.04 batch/s]


(55.27701187133789,
 array([91.74447  , 14.890617 , 99.20127  , 97.93307  , 23.455414 ,
         4.4372635], dtype=float32),
 0.35109996994336445)

In [132]:
train(train_loader, lstm, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)

Using: cuda:0


Epoch 56/1500: 100%|████████████████████| 56/56 [00:04<00:00, 13.36batch/s, counter=49, lastLoss=0.0869, valLoss=0.198]


Early stopping after 56 epochs
Average train loss: 0.003879894217831038
Average validation loss: 0.007644114770823055


([0.29287187568843365,
  0.178091256746224,
  0.1651278090264116,
  0.14924934227019548,
  0.1337069899642042,
  0.12390972727111407,
  0.12023726864052671,
  0.1166123521647283,
  0.11219805252871343,
  0.10824556874909572,
  0.10659497816647802,
  0.10467722572918449,
  0.10426896025559731,
  0.1030503659109984,
  0.10063918706561838,
  0.09829316407974277,
  0.10100247990339994,
  0.1005086772409933,
  0.09933726569371563,
  0.09666824700044734,
  0.09662800362067563,
  0.09771683399698564,
  0.09628901683858462,
  0.09646308462002448,
  0.09634814877063036,
  0.09489081946334668,
  0.0949365645647049,
  0.09567903713988406,
  0.09511102057461228,
  0.09477221979094404,
  0.09292595939976829,
  0.09342557710728475,
  0.09426665412528175,
  0.09209684641765696,
  0.09105379054588932,
  0.09108581367347922,
  0.09174576361796685,
  0.09105341987950462,
  0.08993012005729335,
  0.09307063317724637,
  0.09095588725592409,
  0.08996912304844175,
  0.09073011483997107,
  0.089299566777689

In [120]:
# Remove one FC LAyer
class LSTM(nn.Module):
    def __init__(self, hidden_size=256, single_fc=True, out="f_hidden"):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.out = out
        
        self.cnn = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(128, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        
        self.cnn2 = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            
            nn.Conv1d(128, 64, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        
        self.lstm = nn.LSTM(15, hidden_size, batch_first=True, num_layers=2)
    
        self.fc = nn.Sequential(
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64 , 6),
        )

    def forward(self, x):
        batch_size = x.shape[0]
        x1 = x[:, 0, :].reshape(batch_size, 1, -1)
        x2 = x[:, 1, :].reshape(batch_size, 1, -1)
        
        x1 = self.cnn(x1)
        x2 = self.cnn2(x2)
        
        #x = torch.cat([x1, x2], dim=2)

        
        #h0 = torch.zeros(1, batch_size, 256).to(x.device)
        #c0 = torch.zeros((1, batch_size, self.hidden_size)).to(x.device)
        o1, (h, c) = self.lstm(x1)
        o2, (h2, c) = self.lstm(x2)
        
        
        x = torch.concat([h[-1], h2[-1]], dim=1)
        x = x.reshape(batch_size, -1)
        x = self.fc(x)
        
        return x

In [121]:
ls = LSTM()

In [122]:
train(train_loader, ls, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)

Using: cuda:0


Epoch 1/1500:   0%|                                                                          | 0/56 [00:00<?, ?batch/s]


RuntimeError: input.size(-1) must be equal to input_size. Expected 15, got 32

In [103]:
test(test_loader, ls, scaler)

100%|██████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 15.14 batch/s]


(92.84402561187744,
 array([95.01342, 80.83722, 99.74831, 99.70889, 85.74309, 96.01323],
       dtype=float32),
 0.052071726073821385)